In [116]:
def convert_rcp(filepath: str) -> dict:
    import re

    # Regular expression pattern to extract the RPC values
    pattern = r'(-?\d+(?:\.\d+)?)'

    # Read the RPC values from the text file
    with open(filepath, 'r') as file:
        rpc_values = file.read()

    # Extract the RPC values using the regular expression
    values = [str(x) for x in re.findall(pattern, rpc_values)]

    # Create a dictionary with the keys and values
    rpc_dict = {
        'LINE_OFF': values[0],
        'SAMP_OFF': values[1],
        'LAT_OFF': values[2],
        'LONG_OFF': values[3],
        'HEIGHT_OFF': values[4],
        'LINE_SCALE': values[5],
        'SAMP_SCALE': values[6],
        'LAT_SCALE': values[7],
        'LONG_SCALE': values[8],
        'HEIGHT_SCALE': values[9],
        'LINE_NUM_COEFF': ' '.join(values[10:30]),
        'LINE_DEN_COEFF': ' '.join(values[30:50]),
        'SAMP_NUM_COEFF': ' '.join(values[50:70]),
        'SAMP_DEN_COEFF': ' '.join(values[70:90]),
    	'MIN_LONG': values[91],
        'MIN_LAT': values[92],
        'MAX_LONG': values[93],
        'MAX_LAT': values[94],
        'sampleOFFSET': values[95],
        #'lineOFFSET': values[96],
    }
    return rpc_dict

In [117]:
from osgeo import gdal, osr
import rasterio

def warp_image(image_ds, rpc, output):
    # Create a georeferenced version of the image using the RPCs
    driver = gdal.GetDriverByName('GTiff')
    georef_ds = driver.Create(f'image_georef_{output}.tif', image_ds.RasterXSize, image_ds.RasterYSize, image_ds.RasterCount, gdal.GDT_Byte)
    georef_ds.SetMetadata(rpc, 'RPC')
    georef_ds.GetRasterBand(1).WriteArray(image_ds.GetRasterBand(1).ReadAsArray())
    #print(georef_ds.GetMetadata('RPC'))
    image_ds = None
    georef_ds = None

    # Reproject the images to the same projection (EPSG:3857 in this case)
    # EPSG:4326, EPSG:7789 also not working
    gdal.Warp(output, f'image_georef_{output}.tif')

In [118]:
import cv2
def stitch_images(image_1_path:str, image_2_path:str, outputpath:str):
    # Load the images
    image_1 = cv2.imread(image_1_path)
    image_2 = cv2.imread(image_2_path)

    # Create the stitcher object
    stitcher = cv2.Stitcher.create()

    # Stitch the images together
    status, stitched_image = stitcher.stitch((image_1, image_2))

    # Check if the stitching was successful
    if status == cv2.Stitcher_OK:
        # Save the stitched image
        cv2.imwrite(outputpath, stitched_image)
    else:
        print('Stitching failed!')

In [119]:
import cv2
import numpy as np

# Load the images
image_1 = cv2.imread('/root/PLUS-IBBC/image1_reprojected.tif')
image_2 = cv2.imread('/root/PLUS-IBBC/image2_reprojected.tif')

# Convert the images to grayscale
gray_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
gray_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)

# Set up the block matching algorithm
bm = cv2.StereoBM_create()

print(image_1.shape, image_2.shape)

# Compute the disparity map
disparity = bm.compute(image_1, image_2)

# Estimate the relative depths of the pixels using the RPCs
rpc_1 = convert_rcp('/root/PLUS-IBBC/data/rpc_01.txt')
rpc_2 = convert_rcp('/root/PLUS-IBBC/data/rpc_02.txt')

# Extract the RPC coefficients
line_num_coeff_1 = np.array(rpc_1['LINE_NUM_COEFF'].split(), dtype=float)
line_den_coeff_1 = np.array(rpc_1['LINE_DEN_COEFF'].split(), dtype=float)
samp_num_coeff_1 = np.array(rpc_1['SAMP_NUM_COEFF'].split(), dtype=float)
samp_den_coeff_1 = np.array(rpc_1['SAMP_DEN_COEFF'].split(), dtype=float)

line_num_coeff_2 = np.array(rpc_2['LINE_NUM_COEFF'].split(), dtype=float)
line_den_coeff_2 = np.array(rpc_2['LINE_DEN_COEFF'].split(), dtype=float)
samp_num_coeff_2 = np.array(rpc_2['SAMP_NUM_COEFF'].split(), dtype=float)
samp_den_coeff_2 = np.array(rpc_2['SAMP_DEN_COEFF'].split(), dtype=float)

# Estimate the 3D positions of the points in the left image using the RPCs
num_rows, num_cols = disparity.shape
points_3d_1 = np.empty((num_rows, num_cols, 3))
for i in range(num_rows):
    for j in range(num_cols):
        points_3d_1[i,j,0] = (line_num_coeff_1[0] + line_num_coeff_1[1] * i + line_num_coeff_1[2] * j + line_num_coeff_1[3] * i * j + line_num_coeff_1[4] * i**2 + line_num_coeff_1[5] * j**2) / (line_den_coeff_1[0] + line_den_coeff_1[1] * i + line_den_coeff_1[2] * j + line_den_coeff_1[3] * i * j + line_den_coeff_1[4] * i**2 + line_den_coeff_1[5] * j**2)
        points_3d_1[i,j,1] = (samp_num_coeff_1[0] + samp_num_coeff_1[1] * i + samp_num_coeff_1[2] * j + samp_num_coeff_1[3] * i * j + samp_num_coeff_1[4] * i**2 + samp_num_coeff_1[5] * j**2) / (samp_den_coeff_1[0] + samp_den_coeff_1[1] * i + samp_den_coeff_1[2] * j + samp_den_coeff_1[3] * i * j + samp_den_coeff_1[4] * i**2 + samp_den_coeff_1[5] * j**2)
        points_3d_1[i,j,2] = (points_3d_1[i,j,0] - points_3d_1[i,j,1]) * disparity[i, j]



(2624, 3375, 3) (2706, 3515, 3)


error: OpenCV(4.6.0) /home/conda/feedstock_root/build_artifacts/libopencv_1671461518131/work/modules/calib3d/src/stereobm.cpp:1170: error: (-209:Sizes of input arguments do not match) All the images must have the same size in function 'compute'


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Extract the x, y, and z coordinates of the points
x = points_3d_1[:, :, 0]
y = points_3d_1[:, :, 1]
z = points_3d_1[:, :, 2]

# Create a figure and a 3D Axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the points
ax.scatter(x, y, z)

# Show the plot
plt.show()

In [ ]:
import cv2

def generate_dem(image1_path:str, image2_path:str):
    # Load the images
    image_1 = cv2.imread(image1_path)
    image_2 = cv2.imread(image2_path)

    # Set up the stereo block matching algorithm
    bm = cv2.StereoBM_create()

    # Compute the disparity map
    disparity = bm.compute(image_1, image_2)

    # Convert the disparity map to a 3D point cloud
    points_3d = cv2.reprojectImageTo3D(disparity, stereo_calibration_matrix)

    # Save the point cloud to a file
    np.save('point_cloud.npy', points_3d)

In [129]:
from osgeo import gdal

# Open the first image
ds1 = gdal.Open('image1_reprojected.tif')

# Read the georeferencing information
geotransform = ds1.GetGeoTransform()
srs = osr.SpatialReference()
srs.ImportFromWkt(ds1.GetProjection())

# Get the size of the image in pixels
x_size, y_size = ds1.GetRasterBand(1).GetBlockSize()

# Calculate the extent of the image in the original projection
min_x = abs(geotransform[0])
max_x = abs(geotransform[0]) + abs(x_size * geotransform[1])
min_y = abs(geotransform[3]) + abs(y_size * geotransform[5])
max_y = abs(geotransform[3])
extent_1 = (min_x, max_x, min_y, max_y)

# Open the second image
ds2 = gdal.Open('image2_reprojected.tif')

# Read the georeferencing information
geotransform = ds2.GetGeoTransform()
srs = osr.SpatialReference()
srs.ImportFromWkt(ds2.GetProjection())

# Get the size of the image in pixels
x_size, y_size = ds2.GetRasterBand(1).GetBlockSize()

# Calculate the extent of the image in the original projection
min_x = abs(geotransform[0])
max_x = abs(geotransform[0]) + abs(x_size * geotransform[1])
min_y = abs(geotransform[3]) + abs(y_size * geotransform[5])
max_y = abs(geotransform[3])
extent_2 = (min_x, max_x, min_y, max_y)

# Calculate the intersection of the two extents
min_x = max(extent_1[0], extent_2[0])
max_x = min(extent_1[1], extent_2[1])
min_y = max(extent_1[2], extent_2[2])
max_y = min(extent_1[3], extent_2[3])

# Crop the images to the intersection extent
ds1_cropped = gdal.Translate('image_1_cropped.tif', ds1, projWin=[min_x, max_y, max_x, min_y])
ds2_cropped = gdal.Translate('image_2_cropped.tif', ds2, projWin=[min_x, max_y, max_x, min_y])

# Close the datasets
ds1.Close()
ds2.Close()
ds1_cropped.Close()
ds2_cropped.Close()


ERROR 1: Error: Computed -srcwin 1.56603e+07 -9.2394e+06 0 0 has negative width and/or height.
ERROR 1: Error: Computed -srcwin 1.52514e+07 -8.99275e+06 0 0 has negative width and/or height.


AttributeError: 'Dataset' object has no attribute 'Close'

In [ ]:
from osgeo import gdal, osr

# Open the first image
image_1_ds = gdal.Open('/root/PLUS-IBBC/data/01.tif')

# Get the RPCs for the first image
rpc_1 = convert_rcp("/root/PLUS-IBBC/data/rpc_01.txt")
#print(rpc_1)
# Open the second image
image_2_ds = gdal.Open('/root/PLUS-IBBC/data/02.tif')

# Get the RPCs for the second image
rpc_2 = convert_rcp('/root/PLUS-IBBC/data/rpc_02.txt')

warp_image(image_1_ds,rpc_1,'image1_reprojected.tif')
warp_image(image_2_ds,rpc_2,'image2_reprojected.tif')

image_1 = cv2.imread('image1_reprojected.tif')
print(image_1.shape)
image_1 = cv2.imread('image2_reprojected.tif')
print(image_1.shape)

stitch_images('image1_reprojected.tif','image2_reprojected.tif','stitched.tif')

(2624, 3375, 3)
(2706, 3515, 3)


In [ ]:
# Open the image
image_ds = gdal.Open('data/test1.tif')

# Get the RPCs for the image
rpc = image_ds.GetMetadata('RPC')
print(rpc)

{'ERR_BIAS': '-1', 'ERR_RAND': '-1', 'HEIGHT_OFF': '1295', 'HEIGHT_SCALE': '1315', 'LAT_OFF': '-21.2316081288', 'LAT_SCALE': '0.0911805852907', 'LINE_DEN_COEFF': '1 0.000997771806716 0.000893795146776 -2.56359129684e-05 -1.70851501528e-05 -3.21867506534e-07 2.1532776166e-05 8.13369760723e-05 -0.000270733342464 0.000126355623049 -3.65549547543e-07 1.73348528132e-07 1.0566912918e-05 2.48524027091e-07 5.68010228875e-07 -3.14981737526e-06 -1.46513589459e-07 -1.44200775386e-08 1.59078686184e-06 -3.43796798432e-09', 'LINE_NUM_COEFF': '-37.284870906 -0.389307964671 -39.0126569672 0.756244483967 0.0365724832883 -0.000699590543671 5.69148667027e-05 0.00488795358124 -0.0493487209079 -0.00486335415172 6.61460426948e-05 -4.3251938614e-05 -0.00106436732503 -5.45489448461e-05 -0.00330149225713 -0.0169088089294 -0.00493745513823 6.47041405124e-05 0.000507944645931 9.58883770134e-05', 'LINE_OFF': '19403.5', 'LINE_SCALE': '512', 'LONG_OFF': '55.7119698801', 'LONG_SCALE': '0.0985353286675', 'SAMP_DEN_CO